In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import re
import requests
pd.options.display.max_rows = 5000 

In [2]:
pwd

'/Users/paxton615/Github_Personal/Notes/Notes/Client_stats'

- Stock data: 

spaq : Spinaker Aquisitions

In [3]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/spaq.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

spaq = requests.request("GET", url, headers=headers)

# print(response.text[:500])

- Stock data

pre : Pensana

In [4]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/pre.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

pre = requests.request("GET", url, headers=headers)

In [5]:
spaq = spaq.text
pre = pre.text

In [6]:
def get_dates(string):
    date = re.compile('"date":((.*),)')
    date = date.findall(string)
    date = [i[0][2:12] for i in date]
    date = [datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d") for i in date]
    return date

def get_close(string):
    close = re.compile('"close":((.*),)')
    close = close.findall(string)
    close = [float(i[1].strip()) for i in close]
    return close

In [10]:
spaq_date = get_dates(spaq)
spaq_close = get_close(spaq)

'{\n    "error": "Something happened, please check your parameters and try again!"\n}'

In [8]:
pre_date = get_dates(pre)
pre_close = get_close(pre)

ValueError: time data '07-13-2020' does not match format '%d-%m-%Y'

In [ ]:
print(spaq_date[-1])
print(spaq_close[-1])
print(pre_date[-1])
print(pre_close[-1])

In [ ]:
fund = 10000
client = ['Ma', 'Ma']
stock = ['spaq', 'pre']
total_cap = [10000, 10000]
invest_in = [5000, 5000]
ipo_price = [0.1, 1.34]
w_pirce = [0.2, 0]
s_position = pd.Series(invest_in)/pd.Series(ipo_price)
p_vs_w = [0.5, 0]
w_position = pd.Series(s_position)*pd.Series(p_vs_w)
s_date = [spaq_date[-1], pre_date[-1]]
s_close = [spaq_close[-1]/100, pre_close[-1]/100]

In [ ]:
mr_ma = pd.DataFrame({'client':client, 'stock':stock, 'total_cap':total_cap, 'invest_in':invest_in, \
             'ipo_price':ipo_price, 'w_pirce':w_pirce ,'p_vs_w':p_vs_w,'s_position':s_position, \
             'w_position':w_position, 's_date':s_date, 's_close':s_close,\
             })

In [ ]:
mr_ma['s_pft'] = (mr_ma['s_close'] - mr_ma['ipo_price'])* mr_ma['s_position']

mr_ma['w_pft'] = [(i-j)*k if i>j else 0 for i,j,k in zip(mr_ma.s_close, mr_ma.w_pirce, mr_ma.w_position)]

mr_ma['charge_plcing'] = 0.03 * mr_ma['invest_in'] + 10

In [ ]:
mr_ma

In [ ]:
account_balance = fund + sum(mr_ma['s_pft'])+sum(mr_ma['w_pft'])

total_charges = sum( mr_ma['charge_plcing'])

total_pft = account_balance - total_charges


In [ ]:
print('Account_balance:', account_balance)
print('total_charges:', total_charges)
print('total_profit:', total_pft)


测算一下，如果稳定的拥有30%的年化回报，资金的增长情况是怎样的

In [ ]:
def final_return(initial, rr, times):
    final = initial * ((1+rr)**times)
    return final